In [1]:
from math import ceil, pi, e
from sympy import Symbol, symbols, linear_eq_to_matrix, Matrix, pprint, zeros, solve, pretty, re, N
from termcolor import colored
import numpy as np

In [2]:
def checkint(floatt):
  floatt = float(floatt)
  if floatt%1!=0:
    return floatt
  else:
    return int(floatt)

def truncate(floatt,n=4):
  floatt = str(float(floatt))
  indx = floatt.find('.')
  if indx!=-1:
    try:
      return checkint(floatt[:indx+n+1])
    except:
      return checkint(floatt)
  else:
    return checkint(floatt)

def printt(string,color="red",end='\n'):
  print(colored(string,color),end=end)

In [3]:
def printmtrxs(mtrxs, uni=True):
  rows,cols = mtrxs[0].shape
  if not uni:
    LU = mtrxs[-1]
    mtrxs = mtrxs[:-1]
  for i in range(rows):
    for mtrx in mtrxs:
      print(pretty(mtrx.row(i)),end='\t')
    if not uni:
      print(list(LU.row(i)))
    else:
      print()
  printt('-----------------------------------')

def eqstomatrix(eqs, symbs):
  mtrxs = linear_eq_to_matrix(eqs, symbs)
  A = mtrxs[0]
  b = mtrxs[1]
  lmbda = Matrix(symbs)
  printt('A\t\tX\tb')
  printmtrxs([A,lmbda,b])
  return A,lmbda,b

def getLU(A):
  rows,cols = A.shape
  L, U = list(), list()
  #symbs = list()
  for i in range(1,rows+1):
    row1, row2 = list(), list()
    for j in range(1,cols+1):
      l = Symbol(f'l{i}{j}')
      u = Symbol(f'u{i}{j}')
      #symbs.append(l)
      #symbs.append(u)
      row1.append(l)
      row2.append(u)
    L.append(row1)
    U.append(row2)
  L, U = np.array(L), np.array(U)
  L = np.tril(L,0)
  U = np.triu(U,0)
  return L, U

def LUputvalues(L, U, values):
  L = L.subs(values)
  U = U.subs(values)
  printt('L\t\tU')
  printmtrxs([L,U])
  return L,U

def compare(rhs, lhs, stringg):
  printt(stringg)
  rows,cols = rhs.shape
  values = dict()
  eqs,eqsdone = list(), list()
  length = len(lhs)
  for i in range(length):
    slv = solve(lhs[i]-rhs[i])[-1]
    eqs.append(f'{lhs[i]} = {rhs[i]}')
    if type(slv) is not dict:
      symb = list(lhs[i].free_symbols)[0]
      eqs[i]+= colored(f' => {symb} = {slv}','blue')
      eqsdone.append(i)
      values.update({symb:slv})

  freesymbs = len(lhs.free_symbols)
  while len(values.keys())!=freesymbs:
    for i in range(length):
      if i not in eqsdone:
        slv = solve(lhs[i].subs(values)-rhs[i])
        #printt(f'{lhs[i]} | {lhs[i].subs(values)-rhs[i]} | {slv}','green')
        if len(slv)>0 and type(slv[0]) is not dict:
          slv = slv[-1]
          keys = values.keys()
          vars = list(lhs[i].free_symbols)
          for var in vars:
            if var not in keys:
              values.update({var:slv})
              eqs[i]+= colored(f' => {var} = {slv}','blue')
          eqsdone.append(i)
          break
    #print(values)
          
  [print(eq) for eq in eqs]
  printt('-----------------------------------')
  return values

def LYb(L, b, symbs):
  Y = list()
  for i in range(len(symbs)):
    Y.append(Symbol(f'y{i+1}'))
  Y = Matrix(Y)
  printt('L\t\tY\tb')
  printmtrxs([L,Y,b])
  values = compare(b, Matrix(L.dot(Y)), 'LY = b')
  return values, Y.subs(values)

def UXY(U, Y, symbs):
  x = Matrix(symbs)
  printt('U\t\tX\ty')
  printmtrxs([U,x,Y])
  values = compare(Y, Matrix(U.dot(x)), 'UX = Y')
  return list(x.subs(values))

In [4]:
def doolittle(L, U):
  np.fill_diagonal(L, 1)
  LU = Matrix(L.dot(U))
  L = Matrix(L)
  U = Matrix(U)
  printt('L\t\tU\t\tLU')
  printmtrxs([L,U,LU])
  return L,U,LU

def crout(L, U):
  np.fill_diagonal(U, 1)
  LU = Matrix(L.dot(U))
  L = Matrix(L)
  U = Matrix(U)
  printt('L\t\tU\t\tLU')
  printmtrxs([L,U,LU])
  return L,U,LU

def cholesky(L, U):
  L = U.transpose()
  LU = Matrix(L.dot(U))
  L = Matrix(L)
  U = Matrix(U)
  printt('L\t\tU\t\tLU')
  printmtrxs([L,U,LU], uni=False)
  return L,U,LU

In [5]:
def doolittlemethod(eq, symbs):
  A, lmbda, b = eqstomatrix(eqs, symbs) # equations to matrix
  L, U = getLU(A)                       # get LU from A
  L, U, LU = doolittle(L, U)            # set l11=l22=l22 = 1
  values = compare(A, LU, 'LU = A')     # find values of all variables
  L, U = LUputvalues(L, U, values)      # put values of variables in LU
  printt('AX = b\nA = LU\n(LU)X = b\nL(UX) = b\nlet UX = Y\nthen LY = b',color='green')
  values, Y = LYb(L, b, symbs)          # find Y from LY = b
  xs = UXY(U, Y, symbs)                 # find X from UX = Y
  printt('ANSWER')
  printt(f'{symbs} = {xs}','green')     # print answer

def croutmethod(eq, symbs):
  A, lmbda, b = eqstomatrix(eqs, symbs) # equations to matrix
  L, U = getLU(A)                       # get LU from A
  L, U, LU = crout(L, U)                # set u11=u22=u22 = 1
  values = compare(A, LU, 'LU = A')     # find values of all variables
  L, U = LUputvalues(L, U, values)      # put values of variables in LU
  printt('AX = b\nA = LU\n(LU)X = b\nL(UX) = b\nlet UX = Y\nthen LY = b',color='green')
  values, Y = LYb(L, b, symbs)          # find Y from LY = b
  xs = UXY(U, Y, symbs)                 # find X from UX = Y
  printt('ANSWER')
  printt(f'{symbs} = {xs}','green')     # print answer

def choleskymethod(eq, symbs):
  A, lmbda, b = eqstomatrix(eqs, symbs) # equations to matrix
  L, U = getLU(A)                       # get LU from A
  L, U, LU = cholesky(L, U)             # set U = L transpose
  values = compare(A, LU, 'LU = A')     # find values of all variables
  L, U = LUputvalues(L, U, values)      # put values of variables in LU
  printt('AX = b\nA = LU\n(LU)X = b\nL(UX) = b\nlet UX = Y\nthen LY = b',color='green')
  values, Y = LYb(L, b, symbs)          # find Y from LY = b
  xs = UXY(U, Y, symbs)                 # find X from UX = Y
  printt('ANSWER')
  printt(f'{symbs} = {xs}','green')     # print answer

In [10]:
x,y,z,w = symbols('x y z w')
symbs = [x,y,z,w]
eqs = [4*x + y - z + w + 2,
       x+4*y-z-w+1,
       -x-y+5*z+w,
       x-y+z+3*w-1]

doolittlemethod(eqs, symbs)
#croutmethod(eqs, symbs)
#choleskymethod(eqs, symbs)

A		X	b
[4  1  -1  1]	[x]	[-2]	
[1  4  -1  -1]	[y]	[-1]	
[-1  -1  5  1]	[z]	[0]	
[1  -1  1  3]	[w]	[1]	
-----------------------------------
L		U		LU
[1  0  0  0]	[u₁₁  u₁₂  u₁₃  u₁₄]	[u₁₁  u₁₂  u₁₃  u₁₄]	
[l₂₁  1  0  0]	[0  u₂₂  u₂₃  u₂₄]	[l₂₁⋅u₁₁  l₂₁⋅u₁₂ + u₂₂  l₂₁⋅u₁₃ + u₂₃  l₂₁⋅u₁₄ + u₂₄]	
[l₃₁  l₃₂  1  0]	[0  0  u₃₃  u₃₄]	[l₃₁⋅u₁₁  l₃₁⋅u₁₂ + l₃₂⋅u₂₂  l₃₁⋅u₁₃ + l₃₂⋅u₂₃ + u₃₃  l₃₁⋅u₁₄ + l₃₂⋅u₂₄ + u₃₄]	
[l₄₁  l₄₂  l₄₃  1]	[0  0  0  u₄₄]	[l₄₁⋅u₁₁  l₄₁⋅u₁₂ + l₄₂⋅u₂₂  l₄₁⋅u₁₃ + l₄₂⋅u₂₃ + l₄₃⋅u₃₃  l₄₁⋅u₁₄ + l₄₂⋅u₂₄ + 
l₄₃⋅u₃₄ + u₄₄]	
-----------------------------------
LU = A
u11 = 4 => u11 = 4
u12 = 1 => u12 = 1
u13 = -1 => u13 = -1
u14 = 1 => u14 = 1
l21*u11 = 1 => l21 = 1/4
l21*u12 + u22 = 4 => u22 = 15/4
l21*u13 + u23 = -1 => u23 = -3/4
l21*u14 + u24 = -1 => u24 = -5/4
l31*u11 = -1 => l31 = -1/4
l31*u12 + l32*u22 = -1 => l32 = -1/5
l31*u13 + l32*u23 + u33 = 5 => u33 = 23/5
l31*u14 + l32*u24 + u34 = 1 => u34 = 1
l41*u11 = 1 => l41 = 1/4
l41*u12 + l42*u22 = -1 => l42 = -1/3
l41*u13 + l4

In [7]:
x1, x2, x3 = symbols('x1 x2 x3')
symbs = [x1,x2,x3]
eqs = [2*x1 - 5*x2 + x3 - 12,
       -x1  + 3*x2 - x3 + 8 ,
       3*x1 - 4*x2 + 2*x3 - 16]

#doolittlemethod(eqs, symbs)
#croutmethod(eqs, symbs)
choleskymethod(eqs, symbs)

A		X	b
[2  -5  1]	[x₁]	[12]	
[-1  3  -1]	[x₂]	[-8]	
[3  -4  2]	[x₃]	[16]	
-----------------------------------
L		U		LU
[u₁₁  0  0]	[u₁₁  u₁₂  u₁₃]	[u11**2, u11*u12, u11*u13]
[u₁₂  u₂₂  0]	[0  u₂₂  u₂₃]	[u11*u12, u12**2 + u22**2, u12*u13 + u22*u23]
[u₁₃  u₂₃  u₃₃]	[0  0  u₃₃]	[u11*u13, u12*u13 + u22*u23, u13**2 + u23**2 + u33**2]
-----------------------------------
LU = A
u11**2 = 2 => u11 = sqrt(2)
u11*u12 = -5 => u12 = -5*sqrt(2)/2
u11*u13 = 1 => u13 = sqrt(2)/2
u11*u12 = -1
u12**2 + u22**2 = 3 => u22 = sqrt(38)*I/2
u12*u13 + u22*u23 = -1 => u23 = -3*sqrt(38)*I/38
u11*u13 = 3
u12*u13 + u22*u23 = -4
u13**2 + u23**2 + u33**2 = 2 => u33 = sqrt(627)/19
-----------------------------------
L		U
[√2  0  0]	⎡    -5⋅√2   √2⎤
⎢√2  ──────  ──⎥
⎣      2     2 ⎦	
⎡-5⋅√2   √38⋅ⅈ   ⎤
⎢──────  ─────  0⎥
⎣  2       2     ⎦	⎡   √38⋅ⅈ  -3⋅√38⋅ⅈ ⎤
⎢0  ─────  ─────────⎥
⎣     2        38   ⎦	
⎡√2  -3⋅√38⋅ⅈ   √627⎤
⎢──  ─────────  ────⎥
⎣2       38      19 ⎦	⎡      √627⎤
⎢0  0  ────⎥
⎣       19 ⎦	
--------